In [10]:
# Clone the entire repo.
!git clone -l -s git://github.com/Adithyakaushik326/rock-paper-scissor.git cloned-repo
%cd cloned-repository
!ls

Cloning into 'cloned-repo'...
remote: Enumerating objects: 734, done.
remote: Counting objects: 100% (734/734), done.
remote: Compressing objects: 100% (732/732), done.
remote: Total 734 (delta 0), reused 731 (delta 0), pack-reused 0
Receiving objects: 100% (734/734), 27.06 MiB | 11.83 MiB/s, done.
[Errno 2] No such file or directory: 'cloned-repository'
/content/cloned-repo/cloned-repo
cloned-repo  pic  README.md


In [ ]:
# Fetch a single <1MB file using the raw GitHub URL.
!curl --remote-name \
     -H 'Accept: application/vnd.github.v3.raw' \
     --location https://api.github.com/repos/jakevdp/PythonDataScienceHandbook/contents/notebooks/data/california_cities.csv

In [3]:
import numpy as np
import cv2
import tensorflow as tf
import os
from keras.applications.mobilenet import MobileNet
from keras.models import Sequential,Input,Model
from keras.layers import Dense,Flatten,Dropout,Concatenate,GlobalAveragePooling2D,Lambda,ZeroPadding2D
from keras.layers import SeparableConv2D,BatchNormalization,MaxPooling2D,Conv2D
from keras.preprocessing.image import ImageDataGenerator

In [5]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=45,
        zoom_range=0.2,
        horizontal_flip=False,
        width_shift_range=0.1,  
        height_shift_range=0.1,
        validation_split=0.2,
)

In [11]:
train_gen = train_datagen.flow_from_directory(
    '/content/cloned-repo/cloned-repo/pic/',
    target_size = (160,160),
    batch_size=32,
    class_mode = 'categorical',
    shuffle=True,
)

val_gen = train_datagen.flow_from_directory(
    '/content/cloned-repo/cloned-repo/pic/',
    target_size=(224,224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False,
    subset = 'validation'
)

Found 723 images belonging to 3 classes.
Found 144 images belonging to 3 classes.


In [12]:
train_gen.class_indices

{'paper': 0, 'rock': 1, 'scissor': 2}

In [21]:
model_base = MobileNet(weights='imagenet',include_top=False,input_shape=(160,160,3))
model = Sequential()
model.add(model_base)
model.add(GlobalAveragePooling2D())
model.add(Dropout(0.5))
model.add(Dense(3,activation='softmax'))
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenet_1.00_160 (Function (None, 5, 5, 1024)        3228864   
_________________________________________________________________
global_average_pooling2d_2 ( (None, 1024)              0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 3075      
Total params: 3,231,939
Trainable params: 3,210,051
Non-trainable params: 21,888
_________________________________________________________________


In [22]:
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)
    
history = model.fit_generator(
    train_gen, 
    steps_per_epoch  = 20, 
    validation_data  = val_gen,
    validation_steps = 20,
    epochs = 5, 
    verbose = 2
)

Epoch 1/5
20/20 - 10s - loss: 0.0759 - accuracy: 0.9522 - val_loss: 3.9611 - val_accuracy: 0.3333
Epoch 2/5
20/20 - 7s - loss: 0.0240 - accuracy: 0.9920
Epoch 3/5
20/20 - 7s - loss: 0.0027 - accuracy: 0.9984
Epoch 4/5
20/20 - 7s - loss: 1.8920e-04 - accuracy: 1.0000
Epoch 5/5
20/20 - 7s - loss: 0.0057 - accuracy: 0.9968


In [23]:
model_score = model.evaluate_generator(val_gen,steps=20)
print("Model Test Loss:",model_score[0])
print("Model Test Accuracy:",model_score[1])

Instructions for updating:
Please use Model.evaluate, which supports generators.
Model Test Loss: 0.03970484435558319
Model Test Accuracy: 0.9652777910232544


In [34]:
d = {0:'paper',1:'rock',2:'scissors'}


In [38]:
for i in os.listdir('/content/cloned-repo/cloned-repo/cloned-repo/pic/paper'):
  c = cv2.imread(f'/content/cloned-repo/cloned-repo/cloned-repo/pic/paper/{i}')
  c = c/255.0
  c = np.reshape(c,(-1,160,160,3))
  p = model.predict(c)
  print(p,d[np.argmax(p)])

[[5.6432968e-01 4.3567032e-01 9.5610430e-14]] paper
[[3.9263427e-02 9.6073657e-01 5.1750555e-14]] rock
[[9.8295182e-01 1.7048202e-02 2.9779476e-14]] paper
[[9.2119187e-01 7.8808181e-02 1.1341108e-14]] paper
[[8.8675761e-01 1.1324245e-01 1.2996517e-12]] paper
[[9.8605925e-01 1.3940718e-02 2.6351410e-14]] paper
[[9.4425559e-01 5.5744376e-02 1.6541830e-14]] paper
[[1.9290642e-01 8.0709362e-01 1.4646556e-15]] rock
[[9.9856877e-01 1.4311435e-03 4.3044184e-16]] paper
[[4.6960764e-02 9.5303917e-01 8.6919318e-16]] rock
[[9.277231e-01 7.227688e-02 5.632128e-14]] paper
[[9.9833602e-01 1.6639812e-03 1.5272023e-10]] paper
[[5.9547192e-01 4.0452811e-01 2.9223772e-13]] paper
[[2.9749903e-01 7.0250094e-01 4.9607369e-14]] rock
[[9.9957806e-01 4.2192909e-04 6.2969530e-16]] paper
[[6.2037951e-01 3.7962046e-01 8.7249950e-15]] paper
[[9.0393579e-01 9.6064240e-02 7.6576575e-12]] paper
[[7.5583243e-01 2.4416760e-01 5.4829294e-15]] paper
[[2.9293355e-01 7.0706642e-01 3.9233380e-15]] rock
[[5.3825802e-01 4.61

In [31]:
train_gen.class_indices

{'paper': 0, 'rock': 1, 'scissor': 2}